In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from transformers import ViTFeatureExtractor, ViTForImageClassification,AutoFeatureExtractor,AdamW
from tqdm import tqdm
import torch
import numpy as np
from transformers import ViTImageProcessor, ViTModel
from PIL import Image
import gym
import numpy as np


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pylab as plt


%matplotlib inline

: 

In [ ]:
!pip install psutil

: 

In [ ]:
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224-in21k')

# Load pre-trained feature extractor
feature_extractor = AutoFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")

# Step 1: Modify classification head for the new number of classes
num_classes = 10  # Update with your new number of classes
model.classifier = torch.nn.Linear(model.config.hidden_size, num_classes)
model.config.image_size=32
# Move the model to the desired device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
import gym
from gym import spaces
import numpy as np
import cv2

class CustomVideoEnv(gym.Env):
    def __init__(self, video_path):
        super(CustomVideoEnv, self).__init__()

        # Open the video file
        self.video = cv2.VideoCapture(video_path)

        # Get video properties
        self.width = int(self.video.get(cv2.CAP_PROP_FRAME_WIDTH))
        self.height = int(self.video.get(cv2.CAP_PROP_FRAME_HEIGHT))

        # Set up observation space
        self.observation_space = spaces.Box(low=0, high=255, shape=(self.height, self.width, 3), dtype=np.uint8)

        # Set up action space (assuming discrete actions for simplicity)
        self.action_space = spaces.Discrete(4)  # Four discrete actions: 0, 1, 2, 3

        # Set up initial agent position
        self.agent_pos = [0, 0]

        # Read the first frame
        _, self.current_frame = self.video.read()

        # Set up the viewer (optional, for rendering)
        self.viewer = None

    def reset(self):
        # Reset agent position to the initial state
        self.agent_pos = [0, 0]

        # Read the first frame
        _, self.current_frame = self.video.read()

        # Return the initial observation
        return self._get_observation()

    def step(self, action):
        # Move the agent based on the selected action
        if action == 0:  # Move right
            self.agent_pos[0] = min(self.agent_pos[0] + 1, self.width - 1)
        elif action == 1:  # Move left
            self.agent_pos[0] = max(self.agent_pos[0] - 1, 0)
        elif action == 2:  # Move down
            self.agent_pos[1] = min(self.agent_pos[1] + 1, self.height - 1)
        elif action == 3:  # Move up
            self.agent_pos[1] = max(self.agent_pos[1] - 1, 0)

        # Set the next frame as the new observation
        _, next_frame = self.video.read()

        # Calculate the reward (e.g., negative distance to the target)
        reward = -np.linalg.norm(np.array(self.agent_pos) - np.array([self.width // 2, self.height // 2]))

        # Check if the agent has reached the center (for example)
        done = (self.agent_pos == [self.width // 2, self.height // 2]).all()

        # Update the current frame
        self.current_frame = next_frame

        # Return the observation, reward, done flag, and additional info (optional)
        return self._get_observation(), reward, done, {}

    def _get_observation(self):
        # Resize the current frame to match the observation space
        observation = cv2.resize(self.current_frame, (self.width, self.height))
        return observation

    def render(self, mode='human'):
        # Render the environment (optional)
        if self.viewer is None:
            from gym.envs.classic_control import rendering
            self.viewer = rendering.SimpleImageViewer()

        self.viewer.imshow(self._get_observation())
        return self.viewer.isopen

    def close(self):
        # Close the viewer (optional)
        if self.viewer is not None:
            self.viewer.close()
        # Release the video capture object
        self.video.release()

# Example usage:
video_path = 'path/to/your/video.mp4'
env = CustomVideoEnv(video_path)

# Reset the environment
observation = env.reset()

# Perform some random actions
for _ in range(10):
    action = env.action_space.sample()
    observation, reward, done, _ = env.step(action)
    env.render()

env.close()


In [ ]:
import torch.nn as nn
import torch.optim as optim
import numpy as np
import ptan

# Assuming you have defined CustomVideoEnv
class CustomVideoEnv:
    # ... (implementation details)

# Create an instance of CustomVideoEnv (replace arguments as needed)
custom_env = CustomVideoEnv(video_path='path/to/your/video.mp4')



# Your additional training logic here
class DullModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(DullModel, self).__init__()
        self.fc = nn.Linear(input_size, output_size)
        # Set weights and biases to make the model "dull"
        self.fc.weight.data.fill_(0.0)
        self.fc.bias.data.fill_(0.0)

    def forward(self, x):
        return self.fc(x)

class DullAgent:
    def __init__(self, model, action_space):
        self.model = model
        self.action_space = action_space

    def select_action(self, state):
        q_values = self.model(torch.FloatTensor(state).unsqueeze(0))
        action = np.argmax(q_values.detach().numpy())
        return action,q_values

In [ ]:

# Create a dull neural network model
dull_model = DullModel(custom_env.observation_space.shape[0], custom_env.action_space.n)

# Create a DullAgent
dull_agent = DullAgent(dull_model, action_space=custom_env.action_space.n)

# Define an optimizer
optimizer = optim.Adam(dull_model.parameters(), lr=1e-3)
# Training loop using ExperienceSourceFirstLast
GAMMA = 0.99
MAX_STEPS = 1000
BATCH_SIZE = 32

# Define the experience source
exp_source = ptan.experience.ExperienceSourceFirstLast(custom_env, dull_agent, gamma=GAMMA, steps_count=1)


In [ ]:

# Training loop (just an example, doesn't perform actual training with a dull model)
for _ in range(1000):
    # Generate a fake experience tuple
    state = np.random.random(size=(input_size,))
    action = dull_agent.select_action(state)
    next_state = np.random.random(size=(input_size,))
    reward = 0.0
    done = False

    experience = ptan.experience.ExperienceFirstLast(state, action, reward, last_state=next_state, done=done)

    # Update the dull model (not really updating in this example)
    optimizer.zero_grad()
    loss = torch.nn.functional.mse_loss(dull_model(torch.FloatTensor(state).unsqueeze(0)), torch.zeros(1))
    loss.backward()
    optimizer.step()

In [ ]:

for step, exp in enumerate(exp_source):
    if step > MAX_STEPS:
        break

    # Process the experience tuple
    states, actions, rewards, dones, next_states = exp

    # Convert to PyTorch tensors
    states_v = torch.FloatTensor(states)
    actions_t = torch.LongTensor(actions)
    rewards_v = torch.FloatTensor(rewards)
    dones_mask = torch.BoolTensor(dones)
    next_states_v = torch.FloatTensor(next_states)

    # Compute Q-values for the current and next states
    q_values = dull_model(states_v)
    q_values_for_actions = q_values[range(len(actions)), actions_t]

    # Compute Q-values for the next states
    next_q_values = dull_model(next_states_v)
    next_q_values[dones_mask] = 0.0  # Zero out Q-values for terminal states

    # Compute the target Q-values using the Bellman equation
    expected_q_values = rewards_v + GAMMA * next_q_values.max(dim=1)[0]

    # Calculate the loss
    loss = torch.nn.functional.mse_loss(q_values_for_actions, expected_q_values.detach())

    # Perform the optimization step
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Your additional training logic here

